Read in packages

In [1]:
import pandas as pd
import os
import numpy as np
import xarray as xr
import rioxarray
import glob
import random
import geopandas as gpd
from sklearn.utils import shuffle
from MightyMosaic import MightyMosaic
from sklearn.model_selection import train_test_split



First split the uniqe ids into three sets, based only on fire ids,  I will use these new files to train new models for the anna_good and anna_combined

In [17]:
#path to the good ids
# good_ids = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_training_files.csv')['Files'].tolist()

# good_ids[0]

# df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check_ee.csv')

#path to the original tif files
# tif_path =  '/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85'

#make sure the original tif files are in the good files


#first read in the files which are yes, no or maybe from anna
df = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_poly_check_ee.csv')

df = df.rename(columns = {'Image': 'ID'})

df = df[df['ID'] .isin (['29690000000000-0000000000', '29690000000000-0000023296']) == False]

df['ID'] = df['ID'].astype(int)

# #shapefile which has annas original ids
in_shape = gpd.read_file('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_polygons.shp')

in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)

# Now, convert the GeoDataFrame to a Pandas DataFrame
in_shape = pd.DataFrame(in_shape)
in_shape = in_shape[['Year', 'ID', 'area']]

#left join so I can have year and all info togeter
in_shape = pd.merge(in_shape, df, on = 'ID', how = 'inner')

in_shape['Year'] = in_shape['Year'].astype(int)

# in_shape = in_shape[(in_shape['Year'] == 2019) & (in_shape['Keep'] == 'Yes')]
#filter only keep files
df = in_shape[in_shape['Keep'] == 'Yes']

# in_shape.head()
#for all these ids split into 80% training, 10% validation, 10% testing. 
# Split the dataset into training (80%) and temporary testing (20%)
train_df, temp_test_df = train_test_split(df, test_size=0.2, random_state=42)

# Split the temporary testing dataset into validation (50%) and testing (50%) 
# which corresponds to 10% of the original dataset each
val_df, test_df = train_test_split(temp_test_df, test_size=0.5, random_state=42)

# Save the datasets to CSV files
# train_df.to_csv('train_data.csv', index=False)
# val_df.to_csv('validation_data.csv', index=False)
# test_df.to_csv('test_data.csv', index=False)

print(train_df.shape, val_df.shape, test_df.shape)

train_df.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_training_files_full_fire.csv', index = False)
val_df.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_validation_files_full_fire.csv', index = False)
test_df.to_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files_full_fire.csv', index = False)


(6192, 4) (774, 4) (774, 4)


/tmp/ipykernel_1012788/818267607.py:26: UserWarning: Geometry column does not contain geometry.
  in_shape['geometry'] = in_shape['geometry'].apply(lambda geom: geom.wkt)


Now I need to make new files which will have the specific chunks in them to use for training

In [2]:
training_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_training_files_full_fire.csv')['ID'].tolist()
validation_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/ann_good_validation_files_full_fire.csv')['ID'].tolist()
testing_names = pd.read_csv('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_good_testing_files_full_fire.csv')['ID'].tolist()

# print(training_names[:10])

# #path to the chunked files
# chunked =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_old_subs_0_128') #old is old dnbr method in NA
chunked =  os.listdir('/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_old_subs_0_128')

chunked[:10]

chunked[0].split('_')[-1].split('.')[0]

def filter_chunked(in_names, chunked):
    """
    Filters items in the 'chunked' list based on whether the specified part of
    each item (extracted by splitting the item's string) is in 'training_names'.

    Parameters:
    - training_names: List of integers to filter against.
    - chunked: List of strings, where each string is a filename that contains numbers.

    Returns:
    - List of strings from 'chunked' that match the filtering criteria.
    """
    # Filter the 'chunked' list
    filtered_chunked = [
        name for name in chunked 
        if int(name.split('_')[-1].split('.')[0]) in in_names
    ]
    
    filtered_chunked = ['/explore/nobackup/people/spotter5/cnn_mapping/Russia/anna_training_85_old_subs_0_128/' + i for i in filtered_chunked]
    return filtered_chunked

training_names2 = filter_chunked(training_names, chunked)
validation_names2 = filter_chunked(validation_names, chunked)
testing_names2 = filter_chunked(testing_names, chunked)

print(len(training_names2), len(validation_names2), len(testing_names2))




162808 26919 25027
